Simple implementation of MNIST with TAP-SGM

In [ ]:
import numpy as np
import os
import argparse
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 
from models.unet import UNet

In [ ]:
## FFJORD things
import lib.layers as layers
import lib.utils as utils
import lib.odenvp as odenvp
import lib.multiscale_parallel as multiscale_parallel

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log


In [ ]:
def ffjord_create_model(args, data_shape, regularization_fns):
    hidden_dims = tuple(map(int, args.dims.split(",")))
    strides = tuple(map(int, args.strides.split(",")))

    if args.multiscale:
        model = odenvp.ODENVP(
            (args.batch_size, *data_shape),
            n_blocks=args.num_blocks,
            intermediate_dims=hidden_dims,
            nonlinearity=args.nonlinearity,
            alpha=args.alpha,
            cnf_kwargs={"T": args.time_length, "train_T": args.train_T, "regularization_fns": regularization_fns},
        )
    elif args.parallel:
        model = multiscale_parallel.MultiscaleParallelCNF(
            (args.batch_size, *data_shape),
            n_blocks=args.num_blocks,
            intermediate_dims=hidden_dims,
            alpha=args.alpha,
            time_length=args.time_length,
        )
    else:
        if args.autoencode:

            def build_cnf():
                autoencoder_diffeq = layers.AutoencoderDiffEqNet(
                    hidden_dims=hidden_dims,
                    input_shape=data_shape,
                    strides=strides,
                    conv=args.conv,
                    layer_type=args.layer_type,
                    nonlinearity=args.nonlinearity,
                )
                odefunc = layers.AutoencoderODEfunc(
                    autoencoder_diffeq=autoencoder_diffeq,
                    divergence_fn=args.divergence_fn,
                    residual=args.residual,
                    rademacher=args.rademacher,
                )
                cnf = layers.CNF(
                    odefunc=odefunc,
                    T=args.time_length,
                    regularization_fns=regularization_fns,
                    solver=args.solver,
                )
                return cnf
        else:

            def build_cnf():
                diffeq = layers.ODEnet(
                    hidden_dims=hidden_dims,
                    input_shape=data_shape,
                    strides=strides,
                    conv=args.conv,
                    layer_type=args.layer_type,
                    nonlinearity=args.nonlinearity,
                )
                odefunc = layers.ODEfunc(
                    diffeq=diffeq,
                    divergence_fn=args.divergence_fn,
                    residual=args.residual,
                    rademacher=args.rademacher,
                )
                cnf = layers.CNF(
                    odefunc=odefunc,
                    T=args.time_length,
                    train_T=args.train_T,
                    regularization_fns=regularization_fns,
                    solver=args.solver,
                )
                return cnf

        chain = [layers.LogitTransform(alpha=args.alpha)] if args.alpha > 0 else [layers.ZeroMeanTransform()]
        chain = chain + [build_cnf() for _ in range(args.num_blocks)]
        if args.batch_norm:
            chain.append(layers.MovingBatchNorm2d(data_shape[0]))
        model = layers.SequentialFlow(chain)
    return model

In [ ]:
# import MNIST samples

transform = transforms.Compose([transforms.ToTensor()])
dataset = torchvision.datasets.MNIST('mnist_data',train = True,download = True, transform = transform)



In [ ]:
nrows = 4
ncols = 4
fig = plt.figure(figsize=(8,8))
i = 1
for a in range(nrows):
    for b in range(ncols):
        fig.add_subplot(nrows,ncols,i)
        plt.imshow(dataset[i][0][0],cmap = 'gray')
        i = i+1
        plt.axis('off')


In [ ]:
## Load preconditioning map

inputchannels = 1
inputheight = 28
dimx = inputchannels * inputheight ** 2

checkpoint = torch.load('ffjord_mnist_bad_checkpt.pth',map_location = torch.device('cpu'))
preconditionerargs = checkpoint['args']

data_shape = (inputchannels, inputheight, inputheight)

regularization_fns, regularization_coeffs = create_regularization_fns(preconditionerargs)
model = ffjord_create_model(preconditionerargs,data_shape,regularization_fns)
model.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Scorenet is a U-net, whatever that is. 

## FIGURE OUT SCORENET NEXT

## THEN FIGURE OUT THE PRETRAINED THING

##LOOK AT 2D MARGINALS OF THE BAD FFJORDMAP

## Define scorenetwork 

inputchannels = 1
inputheight = 28
dimx = inputchannels * inputheight ** 2


# scorenet = UNet(input_channels = inputchannels,
#                 input_height = inputheight,
#                 ch = 32,
#                 ch_mult = (1,2,2),
#                 num_res_blocks=2,
#                 attn_resolutions=(16,),
#                 resamp_with_conv=True,
#                 )

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# scorenet = scorenet.to(device)


In [ ]:
depth = 2
hidden_units = dimx * 2 



def construct_score_model(depth,hidden_units):
    chain = []
    chain.append(nn.Linear(int(dimx)+1,int(hidden_units),bias =True))
    chain.append(nn.GELU())

    for ii in range(depth-1):
        chain.append(nn.Linear(int(hidden_units),int(hidden_units),bias = True))
        chain.append(nn.GELU())
    chain.append(nn.Linear(int(hidden_units),dimx,bias = True))    

    return nn.Sequential(*chain)


scorenet =  construct_score_model(depth,hidden_units)
scorenet = scorenet.to(device)

In [ ]:
# Train the scorenet

def calc_loss(score_network: torch.nn.Module, x: torch.Tensor,Tmin,Tmax,eps) -> torch.Tensor:
    # x: (batch_size, nch) is the training data
    
    # sample the time
    t = torch.rand([x.size(0) ]) * (Tmax-Tmin+eps) #+ [1 for _ in range(x.ndim - 1)]).to(x) * (Tmax-Tmin+eps)

    # calculate the terms for the posterior log distribution

    sigmas = torch.sqrt(1 - torch.exp(-t))
    sigmas = sigmas.view(x.shape[0],*([1]*len(x.shape[1:])))
    noise = torch.randn_like(x) * sigmas

    tenlarge = t.repeat(784,1).T
    perturbed_samples = x*torch.exp(-0.5 * tenlarge) + noise

    target = - 1/(sigmas ** 2) * noise
    score_eval_samples = torch.cat((t.reshape(-1,1),perturbed_samples),1)

    scores = -perturbed_samples + score_network(score_eval_samples)

    target = target.view(target.shape[0],-1)
    scores = scores.view(scores.shape[0],-1)
    loss = 0.5 * ((scores-target)**2).sum(dim = -1)

    return loss.mean(dim = 0)

    # int_beta = (0.1 + 0.5 * (20 - 0.1) * t) * t  # integral of beta
    # mu_t = x * torch.exp(-0.5 * int_beta)
    # var_t = -torch.expm1(-int_beta)
    # x_t = torch.randn_like(x) * var_t ** 0.5 + mu_t
    # grad_log_p = -(x_t - mu_t) / var_t  # (batch_size, nch)

    # # calculate the score function
    # score = score_network(x_t, t)  # score: (batch_size, nch)

    # # calculate the loss function
    # loss = (score - grad_log_p) ** 2
    # lmbda_t = var_t
    # weighted_loss = lmbda_t * loss
    # return torch.mean(weighted_loss)

In [ ]:
(torch.randn_like(data)).shape

In [ ]:
## training

loaded = torch.load('pushforward_samples')

scorenet.train()
opt = torch.optim.Adam(scorenet.parameters(),lr = 0.001)

epochs = 100000
for step in range(epochs):

    opt.zero_grad()
    randind = torch.randint(0,59999,(64,))
    data = torch.tensor(loaded[randind,:,:,:])
    data = data.reshape(data.shape[0],-1).to(device)
    # training step
    loss = calc_loss(scorenet, data,0,1,1e-4)
    loss.backward()
    opt.step()
    if not step%100:
        print(loss,step)

In [ ]:
def ou_process(x,T):


    sigmas = torch.sqrt(1 - torch.exp(-torch.tensor(T)))
    noise = torch.randn_like(x) * sigmas

    perturbed_samples = x*torch.exp(-0.5 * T) + noise
    return perturbed_samples


def generate_samples(score_network: torch.nn.Module, nsamples: int,loaded) -> torch.Tensor:
    device = next(score_network.parameters()).device
    x_t = torch.randn_like(loaded[0:16,:,:,:], device=device)  # (nsamples, nch)
    time_pts = torch.linspace(2, 0, 1000, device=device)  # (ntime_pts,)
    # beta = lambda t: 0.1 + (20 - 0.1) * t
    for i in range(len(time_pts) - 1):
        t = time_pts[i]
        dt = time_pts[i + 1] - t

        # calculate the drift and diffusion terms
        fxt = -0.5  * x_t
        gt = 1
        score = -x_t + score_network(x_t, t.expand(x_t.shape[0], 1).squeeze()).detach()
        drift = fxt - gt * gt * score
        diffusion = gt

        # euler-maruyama step
        x_t = x_t - drift * dt + diffusion * torch.randn_like(x_t) * torch.abs(dt) ** 0.5
    return x_t

In [ ]:
loaded = torch.load('pushforward_samples')
loaded = loaded.detach().numpy()

evolve = ou_process(torch.tensor(loaded),torch.tensor(2))


plt.scatter(evolve[0:10000,0,9,1],evolve[0:10000,0,1,23],s = 0.2)
plt.axis('square')


In [ ]:
generated_samps = generate_samples(scorenet_trained,2,torch.tensor(loaded))


In [ ]:
test = model(generated_samps[:,:,:,:],reverse = True)



In [ ]:
# generated_samps = generated_samps.detach().numpy()
plt.scatter(preprocessed[:,0,4,1],preprocessed[:,0,2,2])
plt.scatter(generated_samps[:,0,4,1],generated_samps[:,0,2,2])


In [ ]:
# plt.imshow(test[13,0,:,:].detach().numpy(),cmap = 'gray')

nrows = 4
ncols = 4
fig = plt.figure(figsize=(8,8))
i = 0
for a in range(nrows):
    for b in range(ncols):
        fig.add_subplot(nrows,ncols,i+1)
        plt.imshow(test[i,0,:,:].detach().numpy(),cmap = 'gray')
        i = i+1
        plt.axis('off')


In [ ]:
x = torch.tensor(loaded )
t = torch.rand([x.size(0), ] + [1 for _ in range(x.ndim - 1)]).to(x) * (2)


In [ ]:
preprocessed = torch.load('pushforward_samples').detach().numpy()

In [ ]:
plt.scatter(preprocessed[:,0,19,21],preprocessed[:,0,5,10],s = 0.1)
plt.axis('square')